In [46]:
import numpy as np
import matplotlib.pyplot as plt 
import pickle 
import sys

import os 
sys.path.insert(0, '/../../basic/')
from modules.Utils import running_mean as rm 
from modules.Agents.RepresentationLearning import place_cells
sys.path.insert(0, '../..basic/modules/Agents/RepresentationLearning/')
from Agents.RepresentationLearning.place_cells import place_cells

In [49]:
class place_cells(object):
    def __init__(self, env_shape, num_cells, field_size):
        self.env_shape = env_shape
        self.num_cells = num_cells
        self.field_size = field_size
        self.cell_centres = self.get_cell_centres()

    def get_cell_centres(self):
        cell_centres = []
        for i in range(self.num_cells):
            x, y = np.random.random(2)
            cell_centres.append((x,y))
        return np.asarray(cell_centres)

    # define normalized 2D gaussian
    def gaus2d(self, state):
        x  = state[0]
        y  = state[1]
        mx = self.cell_centres[:,0]
        my = self.cell_centres[:,1]
        sx = sy = self.field_size
        return 1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))

    def get_activities(self, states):
        # TODO - array operation
        activities = []
        for state in states:
            xy_state = (state[1]/self.env_shape[1], state[0]/self.env_shape[0]) # input state as (x, y) in [0,1] i.e. portion of total grid area
            place_cell_activity = self.gaus2d(xy_state)
            activities.append(place_cell_activity)
        return np.asarray(activities)

    def plot_placefields(self, env_states_to_map, num_pc_view=9):
        states = np.asarray(env_states_to_map)
        gridworld_pc_reps = self.get_activities(env_states_to_map)
        get_rand_cells = np.random.choice(self.env_shape[0]*self.env_shape[1],num_pc_view,replace=False)

        num_rows = int(np.ceil(np.sqrt(num_pc_view)))
        num_col = int(np.ceil(np.sqrt(num_pc_view)))
        fig, axes = plt.subplots(num_rows,num_col)

        for i, ax in enumerate(axes.flat):
            if i >= num_pc_view:
                ax.axis('off')
            else:
                # for every state, get what the place cell activity is
                ax.scatter(states[:,1],states[:,0], c =gridworld_pc_reps[:,get_rand_cells[i]])
                cell_center = np.round(np.multiply(self.cell_centres[get_rand_cells[i]],self.env_shape),1)
                ax.set_title(f'{cell_center}')
                ax.set_yticks([])
                ax.set_xticks([])
                ax.invert_yaxis()
                ax.set_aspect('equal')
        #plt.show()
        return get_rand_cells

In [50]:
data_dir = '../Data/results/'

oh = ['c4632834-1950-4487-8a15-14cf6dd4a330',
'73159540-face-484e-8372-49933a5b40bd',
'3712bc3e-75de-4e35-af75-8eed11f7f387',
'bc75125e-446b-43a6-89dc-a35c3003e6e6',
'a8d11e6f-63cc-417d-8d4a-a6429c2f52da']

pc = [
'9f562faa-41dd-415f-afdb-33488309b170',
'86368121-9f72-44c7-a2d5-0b25f2039695',
'd598273d-c994-498d-86cb-9b7fad29aecf',
'93840aff-8148-41ac-a7e0-2d93cde1c820',
'9f8e85c4-c160-4662-9464-b661e510fa47'
]

pc_02 = ['15fc5ae4-20b8-453c-ac42-fa296e7b8754',
'8411546a-ddca-4045-a32f-575f4d5b9ad2',
'5947b44b-ba3d-42e5-8850-54e9b06a95a8',
'969d4b03-4285-4041-ae14-b230712a8dfa',
'0111aa72-4cab-4c0c-941a-12723c196c2c'
]

pc_03 = ['3dd4e397-88a0-478c-b9e0-e4b36553ad00',
'f6faf752-5e6e-48ca-9196-6229ca69c4c7',
'32733a64-5003-4ca4-8ec3-04f4b936d2de',
'888cbd85-6298-4df4-9658-ac359b197d42',
'd357f0a2-dd63-4e15-a225-25c78fb23c7a'
]

big_dict = {'oh':oh, 'pc':pc, 'pc_02':pc_02, 'pc_03':pc_03}

In [53]:
test_reward = pickle.load(open(data_dir+'c4632834-1950-4487-8a15-14cf6dd4a330_data.p','rb'))
print(test_reward.keys())

test_reward = pickle.load(open(data_dir+'5947b44b-ba3d-42e5-8850-54e9b06a95a8_data.p','rb'))['total_reward']

def get_avg_std(list_of_ids):
    results = []
    for id_num in list_of_ids: 
        print(id_num)
        with open(data_dir+ f'{id_num}_data.p', 'rb') as f:
            dats = pickle.loads(f)
            reward_info = dats['total_reward']
            results.append(reward_info)
            
    pp = np.vstack(results)
    
    smoothing = 100
    avg_ = rm(np.mean(pp,axis=0),smoothing)[0:2000]
    std_ = rm(np.std(pp, axis=0), smoothing)[0:2000]
    
    return avg_, std_

dict_keys(['total_reward', 'loss', 'trial_length', 'EC_snap', 'P_snap', 'V_snap'])


ModuleNotFoundError: No module named 'basic'

In [14]:
results_dict = {}
av, st = get_avg_std(pc)
results_dict['pc'] = [av,st]

9f562faa-41dd-415f-afdb-33488309b170


TypeError: a bytes-like object is required, not '_io.BufferedReader'

In [ ]:
import os
print(os.getcwd())

In [ ]:
plt.figure()
plt.plot(avg_, label='OH')
plt.fill_between(np.arange(2000),avg_-std_, avg_+std_, alpha=0.3)